<a href="https://colab.research.google.com/github/redzuanabdullah/Test/blob/main/Journal_of_Business_and_Finance_Librarianship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install bertopic
!pip install scikit-learn
!pip install umap-learn
!pip install hdbscan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files

# Upload the file
uploaded = files.upload()

# After uploading, read the file
import pandas as pd

df = pd.read_csv('scopusJBFL.csv')

# Display the first few rows of the dataframe
df.head()


Saving scopusJBFL.csv to scopusJBFL (1).csv


,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,Millet-Reyes B.,"Millet-Reyes, Benedicte (21741253200)",21741253200,International articles and finance journal ran...,2021,Journal of Business and Finance Librarianship,26.0,3-4,NaN,223,...,NaN,NaN,NaN,English,J. Bus. Financ. Librar.,Article,Final,NaN,Scopus,2-s2.0-85111561672
1,Liu G.,"Liu, Grace (57216258746)",57216258746,"Business students’ experiences, perceptions, a...",2023,Journal of Business and Finance Librarianship,28.0,3,NaN,198,...,NaN,NaN,NaN,English,J. Bus. Financ. Librar.,Article,Final,NaN,Scopus,2-s2.0-85161612952
2,Fadel S.,"Fadel, Stephen (15050445500)",15050445500,Designing a business faculty LibGuide,2021,Journal of Business and Finance Librarianship,26.0,3-4,NaN,235,...,NaN,NaN,NaN,English,J. Bus. Financ. Librar.,Article,Final,NaN,Scopus,2-s2.0-85111815791
3,Van Abel K.,"Van Abel, Kara (57216125709)",57216125709,Business research toolkit: The creation of an ...,2021,Journal of Business and Finance Librarianship,26.0,1-2,NaN,67,...,NaN,NaN,NaN,English,J. Bus. Financ. Librar.,Article,Final,NaN,Scopus,2-s2.0-85106243046
4,Moeller M.C.,"Moeller, Mary Catherine (57222530280)",57222530280,ACCRA cost of living index,2020,Journal of Business and Finance Librarianship,25.0,3-4,NaN,375,...,NaN,NaN,NaN,English,J. Bus. Financ. Librar.,Article,Final,NaN,Scopus,2-s2.0-85103008394


In [ ]:
df.columns

Index(['Authors', 'Author full names', 'Author(s) ID', 'Title', 'Year',
       'Source title', 'Volume', 'Issue', 'Art. No.', 'Page start', 'Page end',
       'Page count', 'Cited by', 'DOI', 'Link', 'Affiliations',
       'Authors with affiliations', 'Abstract', 'Author Keywords',
       'Index Keywords', 'References', 'Correspondence Address', 'Editors',
       'Publisher', 'ISSN', 'ISBN', 'CODEN', 'PubMed ID',
       'Language of Original Document', 'Abbreviated Source Title',
       'Document Type', 'Publication Stage', 'Open Access', 'Source', 'EID'],
      dtype='object')

In [ ]:
# Install required packages
!pip install bertopic
!pip install scikit-learn
!pip install umap-learn
!pip install hdbscan

In [ ]:
# Import necessary libraries
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from nltk.corpus import stopwords
import nltk

# Download stopwords if not already downloaded
nltk.download('stopwords')

# Define minimal custom stopwords - only truly non-meaningful words
minimal_stopwords = {
    # Articles
    'a', 'an', 'the',

    # Conjunctions
    'and', 'but', 'or', 'nor', 'for', 'yet', 'so',

    # Prepositions
    'in', 'to', 'of', 'at', 'on', 'with', 'by', 'from', 'up', 'about', 'into', 'over', 'after',

    # Be verbs and auxiliaries
    'is', 'are', 'was', 'were', 'be', 'been', 'being',
    'have', 'has', 'had', 'having',

    # Pronouns
    'this', 'that', 'these', 'those', 'it', 'its',

    # Common connecting words
    'as', 'than'
}

# Create custom vectorizer with minimal stopwords
vectorizer = CountVectorizer(stop_words=list(minimal_stopwords),
                           ngram_range=(1, 3))

# Create topic model with custom vectorizer
topic_model = BERTopic(
    language="english",
    min_topic_size=2,
    vectorizer_model=vectorizer,
    top_n_words=15,
    verbose=True
)

# Clean and prepare abstracts
abstracts = df['Abstract'].fillna("").astype(str).tolist()

# Fit the model on the abstracts
topics, probs = topic_model.fit_transform(abstracts)

# Get topic information
topic_info = topic_model.get_topic_info()

# Print detailed topic analysis
print("\n=== Top Topics in the Journal ===")
print(topic_info[['Topic', 'Count', 'Name']].head(10))

# Print detailed information about each major topic
print("\n=== Detailed Topic Analysis ===")
for topic in topic_info['Topic'].head(5):
    if topic != -1:  # Skip outlier topic
        print(f"\nTopic {topic}:")
        # Get the top words for this topic
        words = topic_model.get_topic(topic)[:15]
        print("Top words:", ', '.join([word[0] for word in words]))
        print("\nMost representative abstracts:")
        docs = topic_model.get_representative_docs(topic)
        for idx, doc in enumerate(docs[:2]):
            print(f"\nAbstract {idx + 1}:")
            print(doc[:200] + "..." if len(doc) > 200 else doc)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2024-11-16 01:29:58,069 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/24 [00:00<?, ?it/s]

2024-11-16 01:30:36,716 - BERTopic - Embedding - Completed ✓
2024-11-16 01:30:36,719 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-16 01:30:42,856 - BERTopic - Dimensionality - Completed ✓
2024-11-16 01:30:42,858 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-16 01:30:42,915 - BERTopic - Cluster - Completed ✓
2024-11-16 01:30:42,921 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-16 01:30:43,391 - BERTopic - Representation - Completed ✓



=== Top Topics in the Journal ===
   Topic  Count                                               Name
0     -1     63           -1_business_library_librarians_resources
1      0    328  0_available no abstract_abstract available no_...
2      1     59  1_information literacy_information_literacy_st...
3      2     17               2_journal_journals_articles_citation
4      3     14   3_financial_financial literacy_literacy_students
5      4     12        4_library_students_research_research skills
6      5     11  5_transfer_international_inc all rights_press inc
7      6     11   6_distance_services_education_distance education
8      7     10                7_degree_journal_holdings_marketing
9      8      9  8_data_data literacy_literacy_data literacy ed...

=== Detailed Topic Analysis ===

Topic 0:
Top words: available no abstract, abstract available no, available no, no abstract, no abstract available, abstract available, abstract, no, available, , , , , , 

Most representative a

In [ ]:
# Create a DataFrame from the topic information
topic_df = pd.DataFrame({
    'Topic': topic_info['Topic'],
    'Count': topic_info['Count'],
    'Raw_Name': topic_info['Name']
})

# Function to get readable description from topic words
def create_readable_description(topic_num, topic_model):
    if topic_num == -1:
        return "Miscellaneous/Other topics in business librarianship and resources"
    elif topic_num == 0:
        return "Articles pending abstract information"
    else:
        # Get the top words for this topic
        top_words = topic_model.get_topic(topic_num)
        if top_words:
            # Get the top 5 words and their weights
            words = [word[0] for word in top_words[:5]]
            # Create a coherent phrase from the words
            return " ".join(words).replace("_", " ").capitalize()
        else:
            return "No significant terms identified"

# Apply the description function to all topics
topic_df['Description'] = topic_df['Topic'].apply(
    lambda x: create_readable_description(x, topic_model)
)

# Display enhanced topic information
print("\n=== Enhanced Topic Analysis ===")
print("Topic  Count  Description")
print("-" * 80)
for _, row in topic_df.iterrows():
    print(f"{row['Topic']:5d}  {row['Count']:5d}  {row['Description']}")
    # Print top 5 representative documents for each topic
    if row['Topic'] >= 0:  # Skip -1 topic
        print("Representative examples:")
        docs = topic_model.get_representative_docs(row['Topic'])[:2]
        for i, doc in enumerate(docs, 1):
            print(f"  {i}. {doc[:150]}..." if len(doc) > 150 else f"  {i}. {doc}")
        print()

# Optional: Save to CSV for further reference
topic_df.to_csv('detailed_topic_analysis.csv', index=False)


=== Enhanced Topic Analysis ===
Topic  Count  Description
--------------------------------------------------------------------------------
   -1     63  Miscellaneous/Other topics in business librarianship and resources
    0    328  Articles pending abstract information
Representative examples:
  1. [No abstract available]
  2. [No abstract available]

    1     59  Information literacy information literacy students business
Representative examples:
  1. This article examines the new-century business relevance of information literacy and why it is important for undergraduate business students. It then ...
  2. Business leaders have long known that information can provide a significant strategic advantage to companies, and is indeed an important business asse...

    2     17  Journal journals articles citation papers
Representative examples:
  1. A total of 4, 851 bibliographic reference citations from 12 business and economics journals were examined for accuracy to determine if erro